##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Распределенный ввод

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/input"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

API [tf.distribute](https://www.tensorflow.org/guide/distributed_training) предоставляет пользователям простой способ масштабировать обучение модели с одного устройства на несколько. При масштабировании своей модели пользователи также должны распределять входные данные между несколькими устройствами. `tf.distribute` предоставляет API-интерфейс, с помощью которого вы можете автоматически распределять входные данные между устройствами.

Это руководство покажет вам различные способы создания распределенных датасетов и итераторов с помощью `tf.distribute`. Дополнительно будут рассмотрены следующие темы:
- Параметры использования, сегментирования и пакетной обработки при использовании `tf.distribute.Strategy.experimental_distribute_dataset` и `tf.distribute.Strategy.experimental_distribute_datasets_from_function.
- Различные способы итерации по распределенному набору данных.
- Различия между API-интерфейсами `tf.distribute.Strategy.experimental_distribute_dataset`/`tf.distribute.Strategy.experimental_distribute_datasets_from_function` и `tf.data`, а также любые ограничения, с которыми пользователи могут столкнуться при их использовании.

В этом руководстве не рассматривается использование распределенного ввода с помощью Keras API .

## Распределенные датасеты

Для масштабирования данных с помощью API `tf.distribute` рекомендуется использовать объект `tf.data.Dataset`. `tf.distribute` был создан для эффективной работы с `tf.data.Dataset`(например, автоматическая предварительная выборка данных на каждое устройство-ускоритель), при этом в реализацию регулярно включаются оптимизации производительности. Если у вас есть вариант использования чего-то другого, кроме `tf.data.Dataset`, обратитесь к
[разделу](#tensorinputs) в этом руководстве.
В обычном(нераспределенном) цикле обучения, пользователи сначала создают экземпляр `tf.data.Dataset`, а затем перебирают элементы. Например:

In [ ]:
import tensorflow as tf

# Helper libraries
import numpy as np
import os

print(tf.__version__)

In [ ]:
global_batch_size = 16
# Создание объекта tf.data.Dataset.
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

# Итерация по датасету с помощью конструкции for..in...
for inputs in dataset:
  print(train_step(inputs))


Чтобы пользователи могли использовать стратегию `tf.distribute` с минимальными изменениями в существующем коде, были введены два API, которые будут расперделять экземпляр `tf.data.Dataset` и возвращать объект распределенного набора данных. Затем пользователь может итерироваться по распределенному датасету и обучать свою модель, как и раньше. Давайте рассмотрим оба API - `tf.distribute.Strategy.experimental_distribute_dataset` и `tf.distribute.Strategy.experimental_distribute_datasets_from_function` более подробно:

### `tf.distribute.Strategy.experimental_distribute_dataset`

#### Использование

Этот API принимает в качестве входных данных экземпляр `tf.data.Dataset` и возвращает экземпляр `tf.distribute.DistributedDataset`. Вы должны оперделить размер пакета(batch) `tf.data.Dataset` равным глобальному размеру пакета. Глобальный размер пакета - это количество строк данных, которые вы хотите обработать на всех устройствах за 1 шаг. Вы можете итерироваться по распределенному датасету в стиле Python или создать итератор с помощью `iter`. Возвращенный объект не является экземпляром `tf.data.Dataset` и не поддерживает никаких других API, которые каким-либо образом преобразовывают или проверяют набор данных.
Это рекомендуемый API, если у вас нет своего особого способа распределния входных данных по разным устройствам.


In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)
# Распределяем входные данные с помощью `experimental_distribute_dataset`.
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
# 1 глобальный пакет данных, загружаемый в модель за 1 шаг.
print(next(iter(dist_dataset)))

#### Свойства

#####  Разделение на пакеты(Batching)

`tf.distribute` rebatches the input `tf.data.Dataset` instance with a new batch size that is equal to the global batch size divided by the number of replicas in sync. The number of replicas in sync is equal to the number of devices that are taking part in the gradient allreduce during training. When a user calls `next` on the distributed iterator, a per replica batch size of data is returned on each replica. The rebatched dataset cardinality will always be a multiple of the number of replicas. Here are a couple of
examples:

`tf.distribute` обновляет входной экземпляр `tf.data.Dataset`, устанавливая новый размер пакета, равный глобальному размеру пакета, деленному на количество синхронизируемых реплик. Количество синхронизируемых реплик равно количеству устройств, участвующих в уменьшении градиента во время обучения. Когда пользователь вызывает `next` на итераторе распределнного объекта, размер пакета данных для каждой реплики возвращается на каждой реплике. Количество элементов повторно собранного набора данных всегда будет кратным количеству реплик. Вот несколько примеров:

* `tf.data.Dataset.range(6).batch(4, drop_remainder=False)`
  * Без распределения:
    * Batch 1: [0, 1, 2, 3]
    * Batch 2: [4, 5]
  * С распределением на 2 реплики.
    Последний батч ([4, 5]) также распределяется между 2-мя реплтками.

    * Batch 1:
      * Replica 1:[0, 1]
      * Replica 2:[2, 3]
    * Batch 2:
      * Replica 2: [4]
      * Replica 2: [5]



* `tf.data.Dataset.range(4).batch(4)`
  * Без распределения:
    * Batch 1: [[0], [1], [2], [3]]
  * С распределением на 5 реплик:
    * Batch 1:
      * Replica 1: [0]
      * Replica 2: [1]
      * Replica 3: [2]
      * Replica 4: [3]
      * Replica 5: []

* `tf.data.Dataset.range(8).batch(4)`
  * Без распределения:
    * Batch 1: [0, 1, 2, 3]
    * Batch 2: [4, 5, 6, 7]
  * С распределением на 3 реплики:
    * Batch 1:
      * Replica 1: [0, 1]
      * Replica 2: [2, 3]
      * Replica 3: []
    * Batch 2:
      * Replica 1: [4, 5]
      * Replica 2: [6, 7]
      * Replica 3: []

Примечание. Приведенные выше примеры только иллюстрируют, как глобальный пакет разделяется на разные реплики. Не рекомендуется полагаться на фактические значения, которые могут быть на каждой реплике, поскольку они могут меняться в зависимости от реализации.

Повторное объединение набора данных имеет пространственную сложность, которая линейно увеличивается с количеством реплик. Это значит, что в случае использования обучения с несколькими устройствами конвейер ввода может столкнуться с ошибками нехватки памяти.

#####  Масштабирование(Sharding)

`tf.distribute` также выполняет автоматическое масштабирование входного набора данных при распределенном обучении. Каждый набор данных создается на центральном процессоре устройства-воркера. Автомасштабирование набора данных по набору воркеров означает, что каждому воркеру назначается подмножество всего набора данных(если установлена правильная политика масштабирования - `tf.data.experimental.AutoShardPolicy`). Это необходимо для обеспечения того, чтобы на каждом шаге каждый воркер обрабатывал пакет неперекрывающихся элементов набора данных равный размеру глобального пакета. Автомасштабирование имеет несколько различных опций, которые можно указать с помощью `tf.data.experimental.DistributeOptions`.

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
dataset = dataset.with_options(options)

Существует три разных параметра, которые вы можете установить для `tf.data.experimental.AutoShardPolicy`:

* AUTO: это параметр по умолчанию, который означает, что будет предпринята попытка разбить данные типа FILE. Попытка разбиения с помощью политики FILE завершается ошибкой, если не обнаружен файловый набор данных. После этого `tf.distribute` вернется к сегментированию по DATA. Обратите внимание, что если входной набор данных основан на файлах, но количество файлов меньше количества воркеров, будет вызвана ошибка `InvalidArgumentError`. Если это произойдет, явно установите для политики значение `AutoShardPolicy.DATA` или разделите источник ввода на файлы меньшего размера, чтобы количество файлов было больше, чем количество рабочих процессов.
* FILE: выберите это вариант, если хотите разделить входные файлы на всех воркеров. Вы должны использовать эту опцию, если количество входных файлов намного больше, чем количество рабочих, и данные в файлах распределены равномерно. Обратной стороной этого варианта является наличие простаивающих рабочих, если данные в файлах распределены неравномерно. Если количество файлов меньше, чем количество рабочих, будет вызвана ошибка `InvalidArgumentError`. Если это произойдет, явно установите политику на `AutoShardPolicy.DATA`.

В качестве примера давайте распределим 2 файла по 2 воркерам и 2 репликам, по 1 реплике на каждого воркера. Файл 1 содержит [0, 1, 2, 3, 4, 5] и Файл 2 содержит [6, 7, 8, 9, 10, 11]. Пусть общее количество синхронизируемых реплик будет 2, а размер глобального пакета - 4.

  * Worker 0:
    * Batch 1 =  Replica 1: [0, 1]
    * Batch 2 =  Replica 1: [2, 3]
    * Batch 3 =  Replica 1: [4]
    * Batch 4 =  Replica 1: [5]
  * Worker 1:
    * Batch 1 =  Replica 2: [6, 7]
    * Batch 2 =  Replica 2: [8, 9]
    * Batch 3 =  Replica 2: [10]
    * Batch 4 =  Replica 2: [11]

* DATA: Эта политика автомасштабирует элементы для всех воркеров. Каждый из воркеров будет читать весь набор данных и обрабатывать только назначенный ему сегмент. Все остальные сегменты данных будут отброшены. Обычно это используется, если количество входных файлов меньше, чем количество рабочих процессов, и вы хотите улучшить распределение данных по всем рабочим процессам. Обратной стороной является то, что весь набор данных будет прочитан каждым воркером.

В качестве примера давайте распределим 1 файл по 2 воркерам. Файл 1 содержит [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]. Пусть общее количество синхронизируемых реплик равно 2.

  * Worker 0:
    * Batch 1 =  Replica 1: [0, 1]
    * Batch 2 =  Replica 1: [4, 5]
    * Batch 3 =  Replica 1: [8, 9]
  * Worker 1:
    * Batch 1 =  Replica 2: [2, 3]
    * Batch 2 =  Replica 2: [6, 7]
    * Batch 3 =  Replica 2: [10, 11]

* OFF: Если отключить автомасштабирование, то каждый воркер будет обрабатывать все данные.

Давайте распределим 1 файл по 2 воркерам. Файл 1 содержит [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]. Пусть общее количество синхронизируемых реплик равно 2. Тогда каждый воркер увидит следующее распределение:

  * Worker 0:
    * Batch 1 =  Replica 1: [0, 1]
    * Batch 2 =  Replica 1: [2, 3]
    * Batch 3 =  Replica 1: [4, 5]
    * Batch 4 =  Replica 1: [6, 7]
    * Batch 5 =  Replica 1: [8, 9]
    * Batch 6 =  Replica 1: [10, 11]

  * Worker 1:
    * Batch 1 =  Replica 2: [0, 1]
    * Batch 2 =  Replica 2: [2, 3]
    * Batch 3 =  Replica 2: [4, 5]
    * Batch 4 =  Replica 2: [6, 7]
    * Batch 5 =  Replica 2: [8, 9]
    * Batch 6 =  Replica 2: [10, 11] 

##### Предварительная загрузка

По умолчанию `tf.distribute` добавляет вызов метода предварительной выборки в конец экземпляра `tf.data.Dataset`. Аргумент преобразования предварительной выборки, который равен `buffer_size`, равен количеству синхронизируемых реплик.

### `tf.distribute.Strategy.experimental_distribute_datasets_from_function`

#### Использование

Этот API принимает функцию ввода и возвращает экземпляр `tf.distribute.DistributedDataset`. Функция ввода, которую передает пользователь, имеет аргумент `tf.distribute.InputContext` и должна возвращать экземпляр `tf.data.Dataset`. API `tf.distribute` не вносит никаких изменений в пользовательский экземпляр `tf.data.Dataset`, возвращенный функцией ввода. Пользователь несет ответственность за пакетирование и сегментирование набора данных. `tf.distribute` только вызывает переданную функцию ввода на устройстве каждого из воркеров. Помимо того, что пользователи могут указывать свою собственную логику пакетирования и сегментирования, этот API показывает лучшую масштабируемость и производительность по сравнению с `tf.distribute.Strategy.experimental_distribute_dataset`.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def dataset_fn(input_context):
  batch_size = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
  dataset = dataset.shard(
    input_context.num_input_pipelines, input_context.input_pipeline_id)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(2) # Предварительная загрузка 2 пакетов на каждом устройстве.
  return dataset

dist_dataset = mirrored_strategy.experimental_distribute_datasets_from_function(dataset_fn)

#### Свойства

##### Разделение на пакеты(Batching)

Экземпляр `tf.data.Dataset`, который возвращает функция ввода, должен быть упакован с использованием размера пакета для каждой реплики. Размер пакета для каждой реплики - это глобальный размер пакета, разделенный на количество реплик, участвующих в распределенном обучении. Так как `tf.distribute` вызывает функцию ввода на устройстве ЦП каждого из воркеров, набор данных, созданный для данного воркера, должен быть готов к использованию всеми репликами на этом воркере.

##### Масштабирование(Sharding)

Объект `tf.distribute.InputContext`, который неявно передается в качестве аргумента функции ввода пользователя, создается `tf.distribute` под капотом. Он содержит информацию о количестве воркеров, текущем идентификаторе рабочего процесса и т.п. Функция ввода может обрабатывать масштабирование в соответствии с политиками, установленными пользователем в свойствах объекта `tf.distribute.InputContext`.


##### Предварительная загрузка

`tf.distribute` **не** добавляет метод предварительной выборки в конец `tf.data.Dataset`, возвращаемого функцией ввода, предоставленной пользователем.

Примечание:
Оба метода - `tf.distribute.Strategy.experimental_distribute_dataset` и `tf.distribute.Strategy.experimental_distribute_datasets_from_function` возвращают **экземпляры `tf.distribute.DistributedDataset`, которые не относятся к типу `tf.data.Dataset`**. Вы можете итерироваться по этим объектам(как показано в разделе «Распределенные итераторы») и использовать свойство `element_spec`.

## Распределенные итераторы

Подобно нераспределенным объектам `tf.data.Dataset`, вам нужно будет создать итератор для объекта `tf.distribute.DistributedDataset`, чтобы итерироваться по нему и получать доступ к элементам.
Ниже приведены способы, которыми вы можете создать `tf.distribute.DistributedIterator` и использовать его в процессе обучения вашей модели:


### Использование

#### Используйте Python конструкцию цикла for 

Вы можете использовать циклы Python для итерации по `tf.distribute.DistributedDataset`. Элементы, возвращаемые объектом `tf.distribute.DistributedIterator`, могут быть `tf.Tensor` или `tf.distribute.DistributedValues`, содержащие значения для каждой реплики. Размещение цикла внутри `tf.function` даст прирост производительности. Однако в настоящее время `break` и `return` не поддерживаются для циклов по `tf.distribute.DistributedDataset`, который помещается внутри `tf.function`. 
Мы также не поддерживаем размещение цикла внутри `tf.function` при использовании многопользовательских стратегий, таких как `tf.distribute.experimental.MultiWorkerMirroredStrategy` и `tf.distribute.TPUStrategy`. Размещение цикла внутри `tf.function` работает для одного рабочего` tf.distribute.TPUStrategy`, но не при использовании TPU модулей.

In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

for x in dist_dataset:
  # метод train_step обучает модель, используя элементы набора данных
  loss = mirrored_strategy.run(train_step, args=(x,))
  print("Loss is ", loss)

#### Используйте `iter` для создания явного итератора

Чтобы перебирать элементы в экземпляре `tf.distribute.DistributedDataset`, вы можете создать `tf.distribute.DistributedIterator`, используя для него `iter`. С помощью явного итератора вы можете выполнять итерацию для фиксированного количества шагов. Чтобы получить следующий элемент из `tf.distribute.DistributedIterator`(экземпляра `dist_iterator`), вы можете вызвать `next(dist_iterator)`, `dist_iterator.get_next()` или `dist_iterator.get_next_as_optional()`. Первые два по сути одинаковы:

In [ ]:
num_epochs = 10
steps_per_epoch = 5
for epoch in range(num_epochs):
  dist_iterator = iter(dist_dataset)
  for step in range(steps_per_epoch):
    # метод train_step обучает модель, используя элементы набора данных
    loss = mirrored_strategy.run(train_step, args=(next(dist_iterator),))
    # args=(next(dist_iterator),) - тоже самое что и args=(dist_iterator.get_next(),)
    # в таком вызове
    # loss = mirrored_strategy.run(train_step, args=(dist_iterator.get_next(),))
    print("Loss is ", loss)

С помощью `next()` или `tf.distribute.DistributedIterator.get_next()`, если `tf.distribute.DistributedIterator` достиг своего конца, будет выдана ошибка `OutOfRange`. Пользователь может перехватить ошибку на стороне Python и продолжить выполнение других задач, таких как создание контрольных точек и оценка. Однако это не сработает, если вы используете цикл обучения хоста(т.е. Выполняете несколько шагов для каждой `tf.function`), который выглядит так:

```
@tf.function
def train_fn(iterator):
  for _ in tf.range(steps_per_loop):
    strategy.run(step_fn, args=(next(iterator),))
```

 `train_fn` содержит несколько шагов, заключенных в `tf.range`. В этом случае разные итерации цикла могут запускаться параллельно, и ошибка `OutOfRange` может быть вызвана на более поздних итерациях до завершения вычисления предыдущих итераций. При возникновении исключения `OutOfRange` все операции внутри функции будут немедленно завершены, а значит будут прерваны все циклы, даже те, которые не полностью завершились. Если вы хотите избежать таких ошибок, альтернативой, которая не вызывает ошибку `OutOfRange`, является `tf.distribute.DistributedIterator.get_next_as_optional()`. `get_next_as_optional` возвращает `tf.experimental.Optional`, который содержит следующий элемент или не имеет значения(None), если `tf.distribute.DistributedIterator` достиг своего конца. 

In [ ]:
# Вы можете прервать цикл с помощью get_next_as_optional, проверив, содержит ли Optional значение
global_batch_size = 4
steps_per_loop = 5
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "CPU:0"])

dataset = tf.data.Dataset.range(9).batch(global_batch_size)
distributed_iterator = iter(strategy.experimental_distribute_dataset(dataset))

@tf.function
def train_fn(distributed_iterator):
  for _ in tf.range(steps_per_loop):
    optional_data = distributed_iterator.get_next_as_optional()
    if not optional_data.has_value():
      break
    per_replica_results = strategy.run(lambda x:x, args=(optional_data.get_value(),))
    tf.print(strategy.experimental_local_results(per_replica_results))
train_fn(distributed_iterator)

## Использование свойства `element_spec` 

Если вы передаете элементы распределенного набора данных в `tf.function` и хотите получить `tf.TypeSpec`, вы можете указать аргумент `input_signature` функции `tf.function`. Выходные данные распределенного датасета - это `tf.distribute.DistributedValues`, которые могут представлять входные данные для одного или нескольких устройств. Чтобы получить `tf.TypeSpec`, соответствующий этому распределенному значению `tf.distribute.DistributedValues`, вы можете использовать свойство `element_spec` распределенного датасета или распределенного итератора.

In [ ]:
global_batch_size = 16
epochs = 5
steps_per_epoch = 5
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function(input_signature=[dist_dataset.element_spec])
def train_step(per_replica_inputs):
  def step_fn(inputs):
    return 2 * inputs

  return mirrored_strategy.run(step_fn, args=(per_replica_inputs,))

for _ in range(epochs):
  iterator = iter(dist_dataset)
  for _ in range(steps_per_epoch):
    output = train_step(next(iterator))
    tf.print(output)

## Частичные пакеты

Частичные пакеты возникают, когда экземпляры `tf.data.Dataset`, создаваемые пользователями, могут содержать размеры пакетов, которые не делятся без остатка на количество реплик, или когда размер всех данных не делится на размер пакета. В этом случае, когда набор данных распределяется по нескольким репликам, вызов `next` на некоторых итераторах приведет к ошибке `OutOfRangeError`. Для обработки такой ситуации `tf.distribute` возвращает фиктивные пакеты с размером 0 на репликах, у которых больше нет данных для обработки.

Для единичного воркера, если данные не получены вызовом `next` на итераторе, создаются фиктивные пакеты с размером 0, которые используются вместе с реальными данными. В случае частичных пакетов последний глобальный пакет данных будет содержать реальные данные наряду с фиктивными пакетами данных. Обработчик данных теперь проверяет, есть ли данные у какой-либо из реплик. Если данных нет ни на одной из реплик, выдается ошибка `OutOfRange`.

Для случая с несколькими воркерам булево значение, показывающие наличие данных по каждому воркеру, агрегируется с использованием перекрестной связи реплик, и это способ используется, чтобы определить, все ли воркеры завершили обработку распределенного датасета. Поскольку это связано с взаимодействием между воркерам, возникает некоторое снижение производительности.

## Важно!

* При использовании `tf.distribute.Strategy.experimental_distribute_dataset` с несколькими воркерами, пользователи передают `tf.data.Dataset`, который читает из файлов. Если для `tf.data.experimental.AutoShardPolicy` установлено значение `AUTO` или `FILE`, фактический размер пакета для одного шага может быть меньше, чем глобальный размер пакета, определенный пользователем. Это может произойти, когда в файле остается меньше записей чем глобальный размер пакета. Пользователи могут исчерпать набор данных вне зависимости от количества выполняемых шагов или установить для `tf.data.experimental.AutoShardPolicy` значение `DATA`, чтобы избежать этого.

* Преобразования набора данных с отслеживанием состояния в настоящее время не поддерживаются в `tf.distribute`, и любые операции с отслеживанием состояния датасета, в настоящее время игнорируются. Например, если в вашем наборе данных есть `map_fn`, которая использует` tf.random.uniform` для поворота изображения, тогда у вас есть граф набора данных, который зависит от состояния(т.е. случайного начального числа) на локальном компьютере, на котором выполняется процесс Python.

* Экспериментальные опции `tf.data.experimental.OptimizationOptions`, которые отключены по умолчанию, могут в определенных случаях - например, при использовании вместе с` tf.distribute` - вызывать снижение производительности. Их следует включать только после того, как вы убедитесь, что они повышают производительность вашей сети при распределенном обучении.


* Порядок, в котором данные обрабатываются воркерам при использовании `tf.distribute.experimental_distribute_dataset` или `tf.distribute.experimental_distribute_datasets_from_function`, не гарантируется. Обычно порядок выполнения требуется, если вы используете `tf.distribute` для масштабирования прогноза. В таком случае вы можете вставить индекс для каждого элемента в пакете и соответственно упорядочить выходные данные. Следующий фрагмент - пример того, как сортировать выходные данные.

Примечание: в данном случае удобно использовать `tf.distribute.MirroredStrategy()`. Нам нужно изменить порядок входных данных когда мы используем несколько воркеров, а `tf.distribute.MirroredStrategy` используется для обучения по одному воркеру.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
dataset_size = 24
batch_size = 6
dataset = tf.data.Dataset.range(dataset_size).enumerate().batch(batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

def predict(index, inputs):
  outputs = 2 * inputs
  return index, outputs

result = {}
for index, inputs in dist_dataset:
  output_index, outputs = mirrored_strategy.run(predict, args=(index, inputs))
  indices = list(mirrored_strategy.experimental_local_results(output_index))
  rindices = []
  for a in indices:
    rindices.extend(a.numpy())
  outputs = list(mirrored_strategy.experimental_local_results(outputs))
  routputs = []
  for a in outputs:
    routputs.extend(a.numpy())
  for i, value in zip(rindices, routputs):
    result[i] = value

print(result)

<a name="tensorinputs">

### Как мне распределять свои данные, если я не использую tf.data.Dataset?

Иногда пользователи не могут использовать `tf.data.Dataset` для ввода и вышеупомянутые API для распределния датасета на несколько устройств.
В таких случаях вы можете использовать необработанные тензоры или входные данные из генератора.

### Используйте experimental_distribute_values_from_function для произвольных тензорных входов
`strategy.run` принимает` tf.distribute.DistributedValues`, который является возвращаемым значением
`next(iterator)`. Чтобы передать значения тензора, используйте `experimental_distribute_values_from_function` для создания `tf.distribute.DistributedValues` из необработанных тензоров.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
worker_devices = mirrored_strategy.extended.worker_devices

def value_fn(ctx):
  return tf.constant(1.0)

distributed_values = mirrored_strategy.experimental_distribute_values_from_function(value_fn)
for _ in range(4):
  result = mirrored_strategy.run(lambda x:x, args=(distributed_values,))
  print(result)

### Используйте tf.data.Dataset.from_generator если вам нужен вход из генератора

Если у вас есть функция-генератор, которую вы хотите использовать, вы можете создать экземпляр `tf.data.Dataset` с помощью метода `from_generator`.

Примечание. В настоящее время этот метод не поддерживается для `tf.distribute.TPUStrategy`.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
def input_gen():
  while True:
    yield np.random.rand(4)

# используйте Dataset.from_generator
dataset = tf.data.Dataset.from_generator(
    input_gen, output_types=(tf.float32), output_shapes=tf.TensorShape([4]))
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
iterator = iter(dist_dataset)
for _ in range(4):
  mirrored_strategy.run(lambda x:x, args=(next(iterator),))